In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pretty_midi
import librosa
import librosa.display
import gc
from sklearn.preprocessing import StandardScaler
import warnings


import torch
from torch.utils.data import DataLoader, Dataset


from sklearn.preprocessing import StandardScaler

from Preprocessing import *
from ExtractGenre import *

import DatasetLoader as DL

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
InputPath = os.path.realpath('YAMF/genres_original')

GenreMapping = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
          'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

In [48]:
def NormalizeSpectrogram(X):
   X_min = X.min()
   X_max = X.max()
   return (X - X_min) / (X_max - X_min)




def DataCNN(InputPath = os.path.realpath('YAMF/genres_original'), length = 128):

   numErr = 0

   DataList = []
   for dir in tqdm(os.listdir(InputPath)):
      
      DirPath = os.path.join(InputPath, dir)

      if not os.path.isdir(DirPath):
         continue

      genre = GenreMapping[dir]

      for song in os.listdir(DirPath):
         warnings.filterwarnings('ignore')


         SongPath = os.path.join(DirPath, song)

         try:
            y, sr = librosa.load(SongPath, sr=22050)
            mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
            S_db = librosa.amplitude_to_db(mel_spec, ref=np.max)
         except:
            numErr += 1
            continue 

         for _ in range(50):

            rIDX = np.random.randint(0, np.shape(S_db)[1] - length)
            indexs = np.arange(rIDX, rIDX + length)

            X = S_db[:, indexs]

            NormX = NormalizeSpectrogram(X)
            DataList.append((X, NormX))

   return DataList


In [49]:
DataList = DataCNN()

100%|██████████| 11/11 [00:24<00:00,  2.26s/it]


In [50]:
len(DataList)

49950

In [51]:
with open('YAMF/test.pkl', 'wb') as f:
   pickle.dump(DataList, f)